In [136]:
import pandas as pd
import nameparser
import json
import requests
import numpy as np
import uuid

pro_publica_api_key = "PguHImFJakafbsBpZ2ed21A1HIgvXKiM7tg0BqI0 "
headers = {'X-API-Key': pro_publica_api_key}

In [137]:
trans = pd.read_csv('../../data/transactions_20231018.csv')
trans['member'] = trans['member'].str.lower()
trans['parsed_name'] = trans['member'].apply(lambda x: nameparser.HumanName(x))
trans['first_name'] = trans['parsed_name'].apply(lambda x: x.first)
trans['last_name'] = trans['parsed_name'].apply(lambda x: x.last)
trans['string_name'] = trans['first_name'] + ' ' + trans['last_name']
trans['uuid'] = trans.apply(lambda _: uuid.uuid4(), axis=1)
trans['trans_date_copy'] = trans['transaction_date']

trans

,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,member,district,state,...,asset_type,comment,member_type,amount_formatted,parsed_name,first_name,last_name,string_name,uuid,trans_date_copy
0,10/04/2021,2021-09-27,joint,BP,BP plc,purchase,"$1,001 - $15,000",virginia foxx,NC05,NC,...,NaN,NaN,representative,1000.0,virginia foxx,virginia,foxx,virginia foxx,d26ea489-c709-46b1-945f-dee67734f3fe,2021-09-27
1,10/04/2021,2021-09-13,joint,XOM,Exxon Mobil Corporation,purchase,"$1,001 - $15,000",virginia foxx,NC05,NC,...,NaN,NaN,representative,1000.0,virginia foxx,virginia,foxx,virginia foxx,8176d596-bc5f-44e7-8ada-80e9ed20f207,2021-09-13
2,10/04/2021,2021-09-10,joint,ILPT,Industrial Logistics Properties Trust - Common...,purchase,"$15,001 - $50,000",virginia foxx,NC05,NC,...,NaN,NaN,representative,15000.0,virginia foxx,virginia,foxx,virginia foxx,e52a6e63-bf07-48b6-9b73-30a1326c5953,2021-09-10
3,10/04/2021,2021-09-28,joint,PM,Phillip Morris International Inc,purchase,"$15,001 - $50,000",virginia foxx,NC05,NC,...,NaN,NaN,representative,15000.0,virginia foxx,virginia,foxx,virginia foxx,9606c92d-e6eb-4bad-84f0-4ca8037e8226,2021-09-28
4,10/04/2021,2021-09-17,self,BLK,BlackRock Inc,sale_partial,"$1,001 - $15,000",alan s. lowenthal,CA47,CA,...,NaN,NaN,representative,1000.0,alan s. lowenthal,alan,lowenthal,alan lowenthal,5536c9a3-f45b-440b-99d2-b10754dde496,2021-09-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24136,08/17/2012,08/17/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,sheldon whitehouse,NaN,NaN,...,PDF Disclosed Filing,NaN,senator,NaN,sheldon whitehouse,sheldon,whitehouse,sheldon whitehouse,769240b2-1b19-4d92-b827-08e01b497196,08/17/2012
24137,08/16/2012,08/16/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,pat roberts,NaN,NaN,...,PDF Disclosed Filing,NaN,senator,NaN,pat roberts,pat,roberts,pat roberts,4806d662-8bd0-455f-95c4-3c506cb9d547,08/16/2012
24138,08/15/2012,08/15/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,rob portman,NaN,NaN,...,PDF Disclosed Filing,NaN,senator,NaN,rob portman,rob,portman,rob portman,29027e98-0fd3-4929-88f3-29fb630db7b7,08/15/2012
24139,08/02/2012,08/02/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,thomas r. carper,NaN,NaN,...,PDF Disclosed Filing,NaN,senator,NaN,thomas r. carper,thomas,carper,thomas carper,52d2161f-2533-4264-a917-e30f3a6f395e,08/02/2012


In [138]:
trans['transaction_date'] = pd.to_datetime(trans['transaction_date'],format='%Y-%m-%d',errors='coerce')
trans['trans_date_copy'] = pd.to_datetime(trans['trans_date_copy'],format='%m/%d/%Y',errors='coerce')
trans['combined_transaction_date'] = np.where(trans['transaction_date'].isna(), trans['trans_date_copy'], trans['transaction_date'])

trans

,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,member,district,state,...,comment,member_type,amount_formatted,parsed_name,first_name,last_name,string_name,uuid,trans_date_copy,combined_transaction_date
0,10/04/2021,2021-09-27,joint,BP,BP plc,purchase,"$1,001 - $15,000",virginia foxx,NC05,NC,...,NaN,representative,1000.0,virginia foxx,virginia,foxx,virginia foxx,d26ea489-c709-46b1-945f-dee67734f3fe,NaT,2021-09-27
1,10/04/2021,2021-09-13,joint,XOM,Exxon Mobil Corporation,purchase,"$1,001 - $15,000",virginia foxx,NC05,NC,...,NaN,representative,1000.0,virginia foxx,virginia,foxx,virginia foxx,8176d596-bc5f-44e7-8ada-80e9ed20f207,NaT,2021-09-13
2,10/04/2021,2021-09-10,joint,ILPT,Industrial Logistics Properties Trust - Common...,purchase,"$15,001 - $50,000",virginia foxx,NC05,NC,...,NaN,representative,15000.0,virginia foxx,virginia,foxx,virginia foxx,e52a6e63-bf07-48b6-9b73-30a1326c5953,NaT,2021-09-10
3,10/04/2021,2021-09-28,joint,PM,Phillip Morris International Inc,purchase,"$15,001 - $50,000",virginia foxx,NC05,NC,...,NaN,representative,15000.0,virginia foxx,virginia,foxx,virginia foxx,9606c92d-e6eb-4bad-84f0-4ca8037e8226,NaT,2021-09-28
4,10/04/2021,2021-09-17,self,BLK,BlackRock Inc,sale_partial,"$1,001 - $15,000",alan s. lowenthal,CA47,CA,...,NaN,representative,1000.0,alan s. lowenthal,alan,lowenthal,alan lowenthal,5536c9a3-f45b-440b-99d2-b10754dde496,NaT,2021-09-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24136,08/17/2012,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,sheldon whitehouse,NaN,NaN,...,NaN,senator,NaN,sheldon whitehouse,sheldon,whitehouse,sheldon whitehouse,769240b2-1b19-4d92-b827-08e01b497196,2012-08-17,2012-08-17
24137,08/16/2012,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,pat roberts,NaN,NaN,...,NaN,senator,NaN,pat roberts,pat,roberts,pat roberts,4806d662-8bd0-455f-95c4-3c506cb9d547,2012-08-16,2012-08-16
24138,08/15/2012,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,rob portman,NaN,NaN,...,NaN,senator,NaN,rob portman,rob,portman,rob portman,29027e98-0fd3-4929-88f3-29fb630db7b7,2012-08-15,2012-08-15
24139,08/02/2012,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,thomas r. carper,NaN,NaN,...,NaN,senator,NaN,thomas r. carper,thomas,carper,thomas carper,52d2161f-2533-4264-a917-e30f3a6f395e,2012-08-02,2012-08-02


In [139]:
trans['transaction_date'].isna().sum(), trans['trans_date_copy'].isna().sum(), trans['combined_transaction_date'].isna().sum()

(8457, 15695, 11)

In [140]:
names_of_interest = ['david perdue', 'kelly loeffler', 'richard burr']
trans[trans['string_name'].isin(names_of_interest)]

,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,member,district,state,...,comment,member_type,amount_formatted,parsed_name,first_name,last_name,string_name,uuid,trans_date_copy,combined_transaction_date
16997,05/17/2021,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,richard burr,NaN,NaN,...,NaN,senator,NaN,richard burr,richard,burr,richard burr,f71c7463-0507-4c56-b05d-a325880029e3,2021-05-17,2021-05-17
17254,11/04/2020,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,david perdue,NaN,NaN,...,NaN,senator,NaN,david perdue,david,perdue,david perdue,d55dc2ee-29d4-4f37-b6ec-e7bbc25c51b1,2020-11-04,2020-11-04
17421,08/21/2020,NaT,Spouse,ICE,"Intercontinental Exchange, Inc.",Sale (Partial),"$5,000,001 - $25,000,000",kelly loeffler,NaN,GA,...,Equity pursuant to the company he founded with...,senator,5000000.0,kelly loeffler,kelly,loeffler,kelly loeffler,ef510456-61a7-4533-a4c6-4524da6c8963,2020-07-22,2020-07-22
17614,05/22/2020,NaT,Spouse,ICE,"Intercontinental Exchange, Inc.",Purchase,"$1,000,001 - $5,000,000",kelly loeffler,NaN,GA,...,Equity pursuant to the company he founded with...,senator,1000000.0,kelly loeffler,kelly,loeffler,kelly loeffler,7382dc38-488e-4dad-8829-a05b7dd9f058,2020-05-20,2020-05-20
17615,05/22/2020,NaT,Spouse,ICE,"Intercontinental Exchange, Inc.",Sale (Partial),"$5,000,001 - $25,000,000",kelly loeffler,NaN,GA,...,Equity pursuant to the company he founded with...,senator,5000000.0,kelly loeffler,kelly,loeffler,kelly loeffler,76d2aa62-031a-4ac1-b8b2-304704de743f,2020-05-20,2020-05-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24055,02/26/2013,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,richard burr,NaN,NaN,...,NaN,senator,NaN,richard burr,richard,burr,richard burr,c7a35168-be9c-4057-ae8d-fb5d35700106,2013-02-26,2013-02-26
24069,02/06/2013,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,richard burr,NaN,NaN,...,NaN,senator,NaN,richard burr,richard,burr,richard burr,88fedea1-061a-4823-b38e-051b2de09901,2013-02-06,2013-02-06
24079,01/14/2013,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,richard burr,NaN,NaN,...,NaN,senator,NaN,richard burr,richard,burr,richard burr,2424020c-cde8-4ea9-b324-5b6f026f62a6,2013-01-14,2013-01-14
24112,10/19/2012,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,richard burr,NaN,NaN,...,NaN,senator,NaN,richard burr,richard,burr,richard burr,351857d2-1201-4af5-9739-843df633e8bc,2012-10-19,2012-10-19


In [141]:
trans_117 = trans[(trans['combined_transaction_date'] < '2023-01-03') & (trans['combined_transaction_date']  >= '2021-01-03')]
trans_116 = trans[(trans['combined_transaction_date'] < '2021-01-03')]
trans_118 = trans[(trans['combined_transaction_date'] > '2023-01-03')]

trans_117[trans_117['string_name'].isin(names_of_interest)]

,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,member,district,state,...,comment,member_type,amount_formatted,parsed_name,first_name,last_name,string_name,uuid,trans_date_copy,combined_transaction_date
16997,05/17/2021,NaT,NaN,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,richard burr,NaN,NaN,...,NaN,senator,NaN,richard burr,richard,burr,richard burr,f71c7463-0507-4c56-b05d-a325880029e3,2021-05-17,2021-05-17


In [142]:
indices_steube = np.where(trans['string_name'].str.contains('steube'))[0]
for idx in indices_steube:
    trans.at[idx, 'first_name'] = 'greg'

indices_bill = np.where(trans['string_name'].str.contains('william keating'))[0]
for idx in indices_bill:
    trans.at[idx, 'first_name'] = 'bill'

indices_jackson = np.where(trans['string_name'].str.contains('jeff jackson'))[0]
for idx in indices_jackson:
    trans.at[idx, 'first_name'] = 'jeffrey'

indices_flei = np.where(trans['string_name'].str.contains('charles fleischman'))[0]
for idx in indices_flei:
    trans.at[idx, 'first_name'] = 'chuck'

indices = np.where(trans['string_name'].str.contains('david joyce'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'dave'

indices = np.where(trans['string_name'].str.contains('patrick fallon'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'pat'

indices = np.where(trans['string_name'].str.contains('david cawthorn'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'madison'

indices = np.where(trans['string_name'].str.contains('james langevin'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'jim'

indices = np.where(trans['string_name'].str.contains('richard allen'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'rick'

indices = np.where(trans['string_name'].str.contains('bradley schneider'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'brad'

indices = np.where(trans['string_name'].str.contains('aston mceachin'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'a.'

indices = np.where(trans['string_name'].str.contains('david roe'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'phil'

indices = np.where(trans['string_name'].str.contains('lacy clay'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'william'

indices = np.where(trans['string_name'].str.contains('steve chabot'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'steven'

indices = np.where(trans['string_name'].str.contains('joseph morelle'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'joe'

indices = np.where(trans['string_name'].str.contains('linda sanchez'))[0]
for idx in indices:
    trans.at[idx, 'last_name'] = 'sánchez'
trans['string_name'] = trans['first_name'] + ' ' + trans['last_name']

indices = np.where(trans['string_name'].str.contains('bradley schneider'))[0]
for idx in indices:
    trans.at[idx, 'first_name'] = 'brad'

In [143]:
first = {len(trans_116['string_name'].unique())}
second = {len(trans_117['string_name'].unique())}
three = {len(trans_118['string_name'].unique())}

print(f'116 member count {first}' )
print(f'117 member count {second}' )
print(f'118 member count {three}' )

116 member count {174}
117 member count {160}
118 member count {66}


In [144]:
"""JOIN 116th congresses data"""

senate_url = f"https://api.propublica.org/congress/v1/116/senate/members.json"
house_url = f"https://api.propublica.org/congress/v1/116/house/members.json"

senate_116 = requests.get(senate_url, headers=headers)
house_116 = requests.get(house_url, headers=headers)
senate_116_members = senate_116.json()['results'][0]['members']
house_116_members = house_116.json()['results'][0]['members']
senate_116_members = pd.DataFrame(senate_116_members)
house_116_members = pd.DataFrame(house_116_members)
senate_116_members = senate_116_members[['short_title', 'first_name', 'middle_name', 'last_name','party','id']]
house_116_members = house_116_members[['short_title', 'first_name', 'middle_name', 'last_name','party','id']]

members_116 = pd.concat([senate_116_members, house_116_members])
members_116['display_name'] = members_116['short_title'] + ' ' + members_116['first_name']  + ' ' + members_116['last_name']
members_116['member'] = members_116['first_name'] + ' ' + members_116['last_name']
members_116['member'] = members_116['member'].str.lower()
members_116['parsed_name'] = members_116['member'].apply(lambda x: nameparser.HumanName(x))
members_116['first_name'] = members_116['parsed_name'].apply(lambda x: x.first)
members_116['last_name'] = members_116['parsed_name'].apply(lambda x: x.last)
members_116['string_name'] = members_116['first_name'] + ' ' + members_116['last_name']

trans_names_116 = trans_116['string_name'].unique()

members_116 = members_116[members_116['string_name'].isin(trans_names_116)]

trans_116 = trans_116.merge(members_116, how='left', on='string_name')[['display_name',
                                                                        'string_name',

                                                                        'combined_transaction_date',
                                                                        'disclosure_date',
                                                                        'ticker',
                                                                        'asset_description',
                                                                        'type',
                                                                        'amount',
                                                                        'industry',
                                                                        'sector',
                                                                        'state',
                                                                        'ptr_link',

                                                                        'amount_formatted',
                                                                        'id',
                                                                        'uuid'
                                                                        ]]

trans_116

,display_name,string_name,combined_transaction_date,disclosure_date,ticker,asset_description,type,amount,industry,sector,state,ptr_link,amount_formatted,id,uuid
0,Rep. Susie Lee,susie lee,2020-02-24,04/28/2020,BHC,Bausch Health Companies Inc.,purchase,"$1,001 - $15,000",Pharmaceuticals and Biotechnology,Health Care,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,c1c5c603-f2ea-4c99-a9e4-5c158229c0e7
1,Rep. Susie Lee,susie lee,2020-03-06,04/28/2020,BAH,Booz Allen Hamilton Holding Corporation,sale_full,"$1,001 - $15,000",Professional Services,Consumer Services,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,b76332f3-3410-4ed0-82e6-ab19f484a330
2,Rep. Susie Lee,susie lee,2020-03-23,04/28/2020,BYD,Boyd Gaming Corporation,sale_full,"$1,001 - $15,000",Movies/Entertainment,Consumer Discretionary,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,37707207-7c2e-4ff3-9404-ceaeb052b1df
3,Rep. Susie Lee,susie lee,2020-02-04,04/28/2020,CTLT,"Catalent, Inc.",purchase,"$1,001 - $15,000",Biotechnology: Pharmaceutical Preparations,Health Care,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,1eaa4073-8742-4411-8df5-e27ce597a3e6
4,Rep. Susie Lee,susie lee,2020-03-18,04/28/2020,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Real Estate,Finance,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,60f1a149-4c5e-4f06-9ec6-d4c3189e99af
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14465,Sen. Sheldon Whitehouse,sheldon whitehouse,2012-08-17,08/17/2012,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/paper...,NaN,W000802,769240b2-1b19-4d92-b827-08e01b497196
14466,Sen. Pat Roberts,pat roberts,2012-08-16,08/16/2012,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/paper...,NaN,R000307,4806d662-8bd0-455f-95c4-3c506cb9d547
14467,Sen. Rob Portman,rob portman,2012-08-15,08/15/2012,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/paper...,NaN,P000449,29027e98-0fd3-4929-88f3-29fb630db7b7
14468,Sen. Thomas Carper,thomas carper,2012-08-02,08/02/2012,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/paper...,NaN,C000174,52d2161f-2533-4264-a917-e30f3a6f395e


In [145]:
"""JOIN 117th congresses data"""

senate_url = f"https://api.propublica.org/congress/v1/117/senate/members.json"
house_url = f"https://api.propublica.org/congress/v1/117/house/members.json"

senate_117 = requests.get(senate_url, headers=headers)
house_117 = requests.get(house_url, headers=headers)
senate_117_members = senate_117.json()['results'][0]['members']
house_117_members = house_117.json()['results'][0]['members']
senate_117_members = pd.DataFrame(senate_117_members)
house_117_members = pd.DataFrame(house_117_members)
senate_117_members = senate_117_members[['short_title', 'first_name', 'middle_name', 'last_name','party','id']]
house_117_members = house_117_members[['short_title', 'first_name', 'middle_name', 'last_name','party','id']]

members_117 = pd.concat([senate_117_members, house_117_members])
members_117['display_name'] = members_117['short_title'] + ' ' + members_117['first_name']  + ' ' + members_117['last_name']
members_117['member'] = members_117['first_name'] + ' ' + members_117['last_name']
members_117['member'] = members_117['member'].str.lower()
members_117['parsed_name'] = members_117['member'].apply(lambda x: nameparser.HumanName(x))
members_117['first_name'] = members_117['parsed_name'].apply(lambda x: x.first)
members_117['last_name'] = members_117['parsed_name'].apply(lambda x: x.last)
members_117['string_name'] = members_117['first_name'] + ' ' + members_117['last_name']

trans_names_117 = trans_117['string_name'].unique()

members_117 = members_117[members_117['string_name'].isin(trans_names_117)]

trans_117 = trans_117.merge(members_117, how='left', on='string_name')[['display_name',
                                                                        'string_name',

                                                                        'combined_transaction_date',
                                                                        'disclosure_date',
                                                                        'ticker',
                                                                        'asset_description',
                                                                        'type',
                                                                        'amount',
                                                                        'industry',
                                                                        'sector',
                                                                        'state',
                                                                        'ptr_link',

                                                                        'amount_formatted',
                                                                        'id',
                                                                        'uuid'
                                                                        ]]

trans_117

,display_name,string_name,combined_transaction_date,disclosure_date,ticker,asset_description,type,amount,industry,sector,state,ptr_link,amount_formatted,id,uuid
0,Rep. Virginia Foxx,virginia foxx,2021-09-27,10/04/2021,BP,BP plc,purchase,"$1,001 - $15,000",Integrated oil Companies,Energy,NC,https://disclosures-clerk.house.gov/public_dis...,1000.0,F000450,d26ea489-c709-46b1-945f-dee67734f3fe
1,Rep. Virginia Foxx,virginia foxx,2021-09-13,10/04/2021,XOM,Exxon Mobil Corporation,purchase,"$1,001 - $15,000",Integrated oil Companies,Energy,NC,https://disclosures-clerk.house.gov/public_dis...,1000.0,F000450,8176d596-bc5f-44e7-8ada-80e9ed20f207
2,Rep. Virginia Foxx,virginia foxx,2021-09-10,10/04/2021,ILPT,Industrial Logistics Properties Trust - Common...,purchase,"$15,001 - $50,000",Real Estate Investment Trusts,Real Estate,NC,https://disclosures-clerk.house.gov/public_dis...,15000.0,F000450,e52a6e63-bf07-48b6-9b73-30a1326c5953
3,Rep. Virginia Foxx,virginia foxx,2021-09-28,10/04/2021,PM,Phillip Morris International Inc,purchase,"$15,001 - $50,000",Farming/Seeds/Milling,Consumer Non-Durables,NC,https://disclosures-clerk.house.gov/public_dis...,15000.0,F000450,9606c92d-e6eb-4bad-84f0-4ca8037e8226
4,Rep. Alan Lowenthal,alan lowenthal,2021-09-17,10/04/2021,BLK,BlackRock Inc,sale_partial,"$1,001 - $15,000",Investment Bankers/Brokers/Service,Finance,CA,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000579,5536c9a3-f45b-440b-99d2-b10754dde496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8815,Sen. Thomas Carper,thomas carper,2021-01-04,02/01/2021,SH,ProShares Short S&amp;P500,Purchase,"$1,001 - $15,000",NaN,NaN,DE,https://efdsearch.senate.gov/search/view/ptr/a...,1000.0,C000174,404fe6ad-f88b-4e60-a60e-98dbe61c777e
8816,Sen. Sheldon Whitehouse,sheldon whitehouse,2021-01-25,01/25/2021,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/ptr/b...,NaN,W000802,115645d5-6a15-4a8a-baca-08b0bc15e86b
8817,Sen. James Inhofe,james inhofe,2021-01-05,01/15/2021,CRSP,CRISPR Therapeutics AG,Sale (Partial),"$1,001 - $15,000",Biotechnology: Biological Products (No Diagnos...,Health Care,OK,https://efdsearch.senate.gov/search/view/ptr/8...,1000.0,I000024,c4a7734d-86cc-45d4-aaa8-bfd670751450
8818,Sen. Mark Warner,mark warner,2021-01-13,01/13/2021,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/ptr/7...,NaN,W000805,8a59ed72-0306-4a9f-9109-22651fd45ab4


In [147]:
"""JOIN 116th congresses data"""

senate_url = f"https://api.propublica.org/congress/v1/118/senate/members.json"
house_url = f"https://api.propublica.org/congress/v1/118/house/members.json"

senate_118 = requests.get(senate_url, headers=headers)
house_118 = requests.get(house_url, headers=headers)
senate_118_members = senate_118.json()['results'][0]['members']
house_118_members = house_118.json()['results'][0]['members']
senate_118_members = pd.DataFrame(senate_118_members)
house_118_members = pd.DataFrame(house_118_members)
senate_118_members = senate_118_members[['short_title', 'first_name', 'middle_name', 'last_name','party','id']]
house_118_members = house_118_members[['short_title', 'first_name', 'middle_name', 'last_name','party','id']]

members_118 = pd.concat([senate_118_members, house_118_members])
members_118['display_name'] = members_118['short_title'] + ' ' + members_118['first_name']  + ' ' + members_118['last_name']
members_118['member'] = members_118['first_name'] + ' ' + members_118['last_name']
members_118['member'] = members_118['member'].str.lower()
members_118['parsed_name'] = members_118['member'].apply(lambda x: nameparser.HumanName(x))
members_118['first_name'] = members_118['parsed_name'].apply(lambda x: x.first)
members_118['last_name'] = members_118['parsed_name'].apply(lambda x: x.last)
members_118['string_name'] = members_118['first_name'] + ' ' + members_118['last_name']

trans_names_118 = trans_118['string_name'].unique()

members_118 = members_118[members_118['string_name'].isin(trans_names_118)]

trans_118 = trans_118.merge(members_118, how='left', on='string_name')[['display_name',
                                                                        'string_name',
                                                                        'combined_transaction_date',
                                                                        'disclosure_date',
                                                                        'ticker',
                                                                        'asset_description',
                                                                        'type',
                                                                        'amount',
                                                                        'industry',
                                                                        'sector',
                                                                        'state',
                                                                        'ptr_link',
                                                                        'amount_formatted',
                                                                        'id',
                                                                        'uuid'
                                                                        ]]

In [148]:
trans_118

,display_name,string_name,combined_transaction_date,disclosure_date,ticker,asset_description,type,amount,industry,sector,state,ptr_link,amount_formatted,id,uuid
0,Rep. Dan Crenshaw,dan crenshaw,2023-03-20,04/20/2023,SPY,SPDR S7P 500,purchase,"$1,001 - $15,000",NaN,NaN,TX,https://disclosures-clerk.house.gov/public_dis...,1000.0,C001120,3073944c-b649-4dfc-a88a-a12920c45e8f
1,Rep. Josh Gottheimer,josh gottheimer,2023-03-06,04/20/2023,ABBV,AbbVie Inc,purchase,"$1,001 - $15,000",Major Pharmaceuticals,Health Care,NJ,https://disclosures-clerk.house.gov/public_dis...,1000.0,G000583,1b524e36-f623-49c8-9a1a-44286423febe
2,Rep. Josh Gottheimer,josh gottheimer,2023-03-24,04/20/2023,ABBV,AbbVie Inc,sale_partial,"$1,001 - $15,000",Major Pharmaceuticals,Health Care,NJ,https://disclosures-clerk.house.gov/public_dis...,1000.0,G000583,5e3ea1f2-f5ff-470c-a022-2b22a9322b96
3,Rep. Josh Gottheimer,josh gottheimer,2023-03-06,04/20/2023,ALGN,Align Technology Inc,purchase,"$1,001 - $15,000",Industrial Specialties,Health Care,NJ,https://disclosures-clerk.house.gov/public_dis...,1000.0,G000583,8a47232e-6788-473c-891f-c5a8b96c618d
4,Rep. Josh Gottheimer,josh gottheimer,2023-03-06,04/20/2023,GOOG,Alphabet Inc - Class C Capital Stock,sale_partial,"$1,001 - $15,000","Computer Software: Programming, Data Processing",Technology,NJ,https://disclosures-clerk.house.gov/public_dis...,1000.0,G000583,22d8eb9e-5eca-473b-96a6-a03378fb2cbb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,Sen. Thomas Carper,thomas carper,2023-01-11,02/02/2023,FLNG,FLEX LNG Ltd. Ordinary Shares,Purchase,"$1,001 - $15,000",NaN,NaN,DE,https://efdsearch.senate.gov/search/view/ptr/d...,1000.0,C000174,8a87e854-6314-478f-addf-5ff12fb72859
832,Sen. Thomas Carper,thomas carper,2023-01-11,02/02/2023,EQNR,Equinor ASA,Purchase,"$1,001 - $15,000",Integrated oil Companies,Energy,DE,https://efdsearch.senate.gov/search/view/ptr/d...,1000.0,C000174,cb62098f-922a-4b1e-bb6f-9725e1742b65
833,Sen. Thomas Carper,thomas carper,2023-01-11,02/02/2023,MCD,McDonald's Corporation Common Stock,Sale (Full),"$1,001 - $15,000",Restaurants,Consumer Services,DE,https://efdsearch.senate.gov/search/view/ptr/d...,1000.0,C000174,3037eb9c-70c2-4b79-99f3-7024bed1b246
834,Sen. Rick Scott,rick scott,2023-01-19,01/19/2023,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/ptr/6...,NaN,S001217,09100fe3-0102-4f22-8d2b-e65128ac6372


In [149]:
trans_116['congress'] = 116
trans_117['congress'] = 117
trans_118['congress'] = 118

In [150]:
transactions = pd.concat([trans_116, trans_117, trans_118], axis=0, ignore_index=True, sort=False)

transactions[transactions['string_name'].isin(names_of_interest)]

,display_name,string_name,combined_transaction_date,disclosure_date,ticker,asset_description,type,amount,industry,sector,state,ptr_link,amount_formatted,id,uuid,congress
7583,Sen. David Perdue,david perdue,2020-11-04,11/04/2020,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/ptr/a...,NaN,P000612,d55dc2ee-29d4-4f37-b6ec-e7bbc25c51b1,116
7750,Sen. Kelly Loeffler,kelly loeffler,2020-07-22,08/21/2020,ICE,"Intercontinental Exchange, Inc.",Sale (Partial),"$5,000,001 - $25,000,000",Investment Bankers/Brokers/Service,Finance,GA,https://efdsearch.senate.gov/search/view/ptr/f...,5000000.0,L000594,ef510456-61a7-4533-a4c6-4524da6c8963,116
7943,Sen. Kelly Loeffler,kelly loeffler,2020-05-20,05/22/2020,ICE,"Intercontinental Exchange, Inc.",Purchase,"$1,000,001 - $5,000,000",Investment Bankers/Brokers/Service,Finance,GA,https://efdsearch.senate.gov/search/view/ptr/c...,1000000.0,L000594,7382dc38-488e-4dad-8829-a05b7dd9f058,116
7944,Sen. Kelly Loeffler,kelly loeffler,2020-05-20,05/22/2020,ICE,"Intercontinental Exchange, Inc.",Sale (Partial),"$5,000,001 - $25,000,000",Investment Bankers/Brokers/Service,Finance,GA,https://efdsearch.senate.gov/search/view/ptr/c...,5000000.0,L000594,76d2aa62-031a-4ac1-b8b2-304704de743f,116
7945,Sen. Kelly Loeffler,kelly loeffler,2020-05-20,05/22/2020,ICE,"Intercontinental Exchange, Inc.",Sale (Partial),"$1,000,001 - $5,000,000",Investment Bankers/Brokers/Service,Finance,GA,https://efdsearch.senate.gov/search/view/ptr/c...,1000000.0,L000594,66a9fcfd-434c-4f40-9182-38f9894e79bc,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14398,Sen. Richard Burr,richard burr,2013-02-06,02/06/2013,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/paper...,NaN,B001135,88fedea1-061a-4823-b38e-051b2de09901,116
14408,Sen. Richard Burr,richard burr,2013-01-14,01/14/2013,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/paper...,NaN,B001135,2424020c-cde8-4ea9-b324-5b6f026f62a6,116
14441,Sen. Richard Burr,richard burr,2012-10-19,10/19/2012,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/paper...,NaN,B001135,351857d2-1201-4af5-9739-843df633e8bc,116
14456,Sen. Richard Burr,richard burr,2012-09-11,09/11/2012,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/paper...,NaN,B001135,5409c02b-2e30-4b6f-b48e-518e9f45814f,116


In [151]:
transactions.to_csv('../prod_data/transactions.csv', index=False)

In [100]:
id = transactions[['id','congress']].drop_duplicates()
id

,id,congress
0,L000590,116
31,L000579,116
34,B001274,116
36,W000804,116
44,D000617,116
...,...,...
24000,S001203,118
24021,M000355,118
24024,S001198,118
24030,B001236,118


In [101]:
transactions_old = pd.read_csv('../prod_data/transactions.csv')
old_id = transactions_old[['id','congress']].drop_duplicates()

old_id

,id,congress
0,L000590,116
31,L000579,116
34,B001274,116
36,W000804,116
44,D000617,116
...,...,...
15454,D000628,118
15545,J000308,118
15577,T000488,118
15654,C001078,118


In [106]:
merged_df = id.merge(old_id, on=['id','congress'], how='outer', indicator=True)


new_ids = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])
new_ids.to_csv('../prod_data/new_ids.csv')

In [104]:
t

,display_name,transaction_date,disclosure_date,ticker,asset_description,type,amount,industry,sector,state,amount_formatted,id,uuid,congress
